<a href="https://colab.research.google.com/github/Anjasfedo/Learning-TensorFlow/blob/main/eat_tensorflow2_in_30_days/Chapter4_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 4-3 Rules of Using the AutoGraph

There are three ways of constructing graph:
1. static
2. dynamic
3. autograph

TensorFlow 2.X uses dynamic graph and autograph.

Dynamic graph is easier for debugging with higher encoding efficiency, byt with lower efficiency in execution.

Static graph has high efficiency in execution, but more difficult for debugging.

Autograph mechanism transform dynamic graph into static graph, making allowance for both executing and encoding efficiencies.

There are certain rules for the code that is able to converted by Autograph, or could result in failure or unexpected results.

Here is the introduce the coding rules of Autograph and its mechanism of converting into static graph, together with introduction about how to construct Autograph using `tf.Module`.

## 1. Summarization of the Coding Rules of Autograph
- i. We should use TensorFlow-defined functions to be decoradeb by `@tf.function` as much as possible, instead of those Python functions. For instance, `tf.print` should be used of print; `tf.range` should be used instead of `range`; `tf.constant(True)` shoud be use instead of `True`.
- ii. Avoid defining `tf.variable` inside the decorator `@tf.function`.
- iii. Functions that are decorated by `@tf.function` cannot modify the struct data types variables outside the function such as Python list, dictionary, etc.

## 2. Explanations to the Autograph Coding Rules

### 2.1 We should use the TensorFlow-defined functions to be decorated by `@tf.function` as much as possible, instead of those Python functions.

In [10]:
import numpy as np
import tensorflow as tf

@tf.function
def np_random():
    a = np.random.randn(3, 3)
    tf.print(a)

@tf.function
def tf_random():
    a = tf.random.normal([3, 3])
    tf.print(a)

In [11]:
# Same results after each execution of np_random
np_random()
np_random()

array([[-1.37656135,  0.30094502,  1.77331609],
       [-0.42099093,  0.50489307,  0.95303633],
       [-0.91528498, -1.11160235, -1.49969172]])
array([[-1.37656135,  0.30094502,  1.77331609],
       [-0.42099093,  0.50489307,  0.95303633],
       [-0.91528498, -1.11160235, -1.49969172]])


In [12]:
# New random numbers are generated after each execution of tf_random
tf_random()
tf_random()

[[-0.30629158 0.820260286 1.89050579]
 [-0.144629896 0.945070803 -2.69782138]
 [-0.376780778 0.641351104 -0.600236237]]
[[0.986099243 -0.676460862 2.05970693]
 [0.572576046 -1.23816967 1.31713629]
 [-0.23802799 -0.637733638 1.34388292]]


### 2.2 Avoid defining `tf.Variable` inside the decorator `@tf.function`.

In [13]:
x = tf.Variable(1.0, dtype=tf.float32)

@tf.function
def outer_var():
  x.assign_add(1.0)
  tf.print(x)

outer_var()
outer_var()

2
3


In [14]:
@tf.function
def inner_var():
  x = tf.Variable(1.0, dtype=tf.float32)
  x.assign_add(1.0)
  tf.print(x)

# Cause an error
inner_var()
inner_var()

ValueError: in user code:

    File "<ipython-input-14-caeae27be669>", line 3, in inner_var  *
        x = tf.Variable(1.0, dtype=tf.float32)

    ValueError: tf.function only supports singleton tf.Variables created on the first call. Make sure the tf.Variable is only created once or created outside tf.function. See https://www.tensorflow.org/guide/function#creating_tfvariables for more information.


### 2.3 Functions that are decorated by `@tf.function` cannot modify the struct data types variables outside the function such as Python list, dictionary, etc.

In [21]:
tensor_list = []

def append_tensor(x):
    tensor_list.append(x)
    return tensor_list

append_tensor(tf.constant(5.0))
append_tensor(tf.constant(6.0))
print(tensor_list)

[<tf.Tensor: shape=(), dtype=float32, numpy=5.0>, <tf.Tensor: shape=(), dtype=float32, numpy=6.0>]


In [22]:
tensor_list = []

@tf.function # Autograph will result in something unexpected if executing this line
def append_tensor(x):
    tensor_list.append(x)
    return tensor_list

append_tensor(tf.constant(5.0))
append_tensor(tf.constant(6.0))
print(tensor_list)

[<tf.Tensor 'x:0' shape=() dtype=float32>]
